In [1]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 618.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 807.4 kB/s eta 0:00:00 0:00:01m


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 631.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 753.8 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 711.5 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 661.9 kB/s eta 0:00:000:00:01
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=0807c5913e180ca9ca381af2dddfb2ed0ac0b188a949b75426450f7ebec38572
  Stored in directory: /Users/kaidixu/Library/Caches/pip/wheels/75/a3/1a/2f3f90b9a4eb0408109ae1b5bae01efbdf8ab4ef98797433e4
Successfully built ffmpy
 

In [2]:
import hopsworks
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194726
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Load traffic and weather datasets
traffic_train = pd.read_csv('traffic_trainset_hour.csv')
weather_train = pd.read_csv('weather_trainset.csv')

In [5]:
# Insert DataFrames into Feature Groups
traffic_fg = fs.create_feature_group(name='traffic_feature_group',
                                     version=1,
                                     description='Feature group for traffic data',
                                     primary_key=['day', 'hour'])

weather_fg = fs.create_feature_group(name='weather_feature_group',
                                     version=1,
                                     description='Feature group for weather data',
                                     primary_key=['day', 'hour'])

traffic_fg.insert(traffic_train)
weather_fg.insert(weather_train)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194726/fs/194645/fg/376151


Uploading Dataframe: 0.00% |          | Rows 0/48 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: traffic_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/194726/jobs/named/traffic_feature_group_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194726/fs/194645/fg/377168


Uploading Dataframe: 0.00% |          | Rows 0/48 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/194726/jobs/named/weather_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc940129580>, None)

In [8]:
# Join traffic and weather data in feature groups on [day, hour]
merged_data = traffic_fg.select_all().join(weather_fg.select_all(), on=['day', 'hour'])

In [9]:
# Create a feature view for the merged data
merged_fv = fs.get_or_create_feature_view(
    name='merged_trainset',
    query=merged_data,
    version=1,
    labels=['current_speed']
)


Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194726/fs/194645/fv/merged_trainset/version/1
